<h1><center> ENSF 519.01 Applied Data Scince </center></h1>
<h2><center> Final Project </center></h2>
<h3><center> Oscar Predictions from Movie Scripts </center></h3>
<p><center> Mark Ricalde, Paul Chen, Brian Pho, Harjee Johal <center></p>

# Phase 1

## Data Preprocessing

## Visualization and Exploratory Data Analysis

# Phase 2

## Discriminative Models

## Generative Models

## Clustering Models

## Machine Learning Models

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# https://www.tensorflow.org/guide/keras/overview

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# Add another:
model.add(layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
layers.SimpleRNN()
layers.GRU()
layers.LSTM()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

def clean_text(text):
    #Lower casing the text
    text = text.lower()
    
    #Removing punctuation
    translation = text.maketrans("", "", string.punctuation)
    text = text.translate(translation)
    re.sub("\s\s+", " ", text)
    
    #Removing stop words
    text = " ".join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])
    
    #Stemming the words
    text = " ".join([stemmer.stem(word) for word in text.split()])
    
    return text

#Cleaning the script
data["script"] = data["script"].apply(clean_text)

vectors = data["text"]
labels = sms["label"]
   
#Splitting the data set    
X_train, X_test, y_train, y_test = train_test_split(vectors, labels, stratify=labels)

#Vectorizing the training set
tfidf_vect = TfidfVectorizer().fit(X_train)
X_train = tfidf_vect.transform(X_train)

#Logistic Regression
logreg = LogisticRegression(solver="liblinear").fit(X_train, y_train)
logreg.score(X_test, y_test)